### Training

In [ ]:
# Tải thư viện ultralytics
%pip install -U ultralytics

# Remember restart kernal after install
# Using numpy version 1.26.4 to running RT_DERT
!pip install -U "numpy<2" --force-reinstall

In [ ]:
# Create .yaml file to running DERT

WIN_DATA_ROOT = r"/data/bdd100k/BDD100k"
WIN_YAML_PATH = r"/workspace/bdd100k_dert.yaml"
RUNS_ROOT     = r"/workspace/runs/detect_dert"

# Tạo file YAML (nếu bạn đã có sẵn file đúng nội dung thì có thể bỏ qua cell này)
yaml_text = f"""path: {WIN_DATA_ROOT}
train: images\\train
val: images\\val
names:
  0: person
  1: rider
  2: car
  3: truck
  4: bus
  5: train
  6: motor
  7: bike
  8: traffic light
  9: traffic sign
"""

import os
os.makedirs(os.path.dirname(WIN_YAML_PATH), exist_ok=True)
with open(WIN_YAML_PATH, "w", encoding="utf-8") as f:
    f.write(yaml_text)

print("YAML saved to:", WIN_YAML_PATH)
print("Data root     :", WIN_DATA_ROOT)
print("Runs root     :", RUNS_ROOT)


In [ ]:
from ultralytics import YOLO

device = 0 if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())

model = YOLO("rtdetr-l.pt")  # tải trọng số COCO sẵn có
results = model.train(
    data=WIN_YAML_PATH,
    epochs=50,           # 50
    imgsz=640,
    batch=2,             # 64
    device=device,           
    workers=4,           # 4
    cache=True,         # True
    close_mosaic=10,
    save_period=5,
    deterministic=False,
    amp = True,         # True
    # pretrained=True,
    project=RUNS_ROOT,   # thư mục lưu runs
    patience=0,  
    name="bdd100k_rtdetr_l"
)

print("Done training. Best model should be at:")
print(os.path.join(RUNS_ROOT, "bdd100k_rtdetr_l", "weights", "best.pt"))
